In [ ]:
from pathlib import Path

In [ ]:
import sys
sys.path.insert(0, "/share/home/ychi/dev")

In [ ]:
ddir = Path("/shareb/ychi/repo/sperm_struct/notebooks/D_primary_views_pileup/")

In [ ]:
from hic_basic.hicio import read_meta

In [ ]:
meta = read_meta("../A_meta/meta/tillsperm30.meta.csv.gz")

In [ ]:
gs = meta.query(
    '(strain == "B6") & (condition == "normal")'
    )["20k_g_struct1"].dropna()

In [ ]:
from hires_utils.hires_utils.hires_io import parse_3dg
from hic_basic.structure.measure import primary_views

In [ ]:
import pickle
from functools import partial

In [ ]:
Path(gs[0]).with_suffix(".pkl").name

In [ ]:
outfile = ddir / "primary_views.pkl"
Path(ddir/"cache").mkdir(exist_ok=True)

def get_primary_views(fili, ngrid, cache_dir=None):
    if cache_dir:
        if (cache_dir / Path(fili).with_suffix(".pkl").name).exists():
            print("Loading %s from cache." % fili)
            with open(cache_dir+Path(fili).stem+".pkl", "rb") as f:
                return pickle.load(f)
    data = parse_3dg(fili)
    print("Parsed %s" % fili)
    res = primary_views(data, ngrid, method="ray")
    print("Processing %s done." % fili)
    if cache_dir:
        with open((cache_dir / Path(fili).with_suffix(".pkl").name), "wb") as f:
            pickle.dump(res, f)
    return res
print("Start processing...")
ares = list(map(partial(get_primary_views, ngrid=32, cache_dir=ddir / "cache"), gs))
print("All done.")
with open(outfile, "wb") as f:
    pickle.dump(dict(zip(gs.index,ares)), f)